<a href="https://colab.research.google.com/github/AbdulRehman2106/Python-Assignment-4/blob/main/BMI_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import streamlit as st
import pandas as pd
from fpdf import FPDF

st.set_page_config(page_title="Advanced BMI Calculator", page_icon="💪", layout="centered")
st.title("💪 Advanced BMI Calculator")

if "history" not in st.session_state:
    st.session_state.history = []

unit = st.radio("Select unit system:", ["Metric (kg/m)", "Imperial (lbs/ft-in)"], horizontal=True)

if unit == "Metric (kg/m)":
    st.subheader("Enter your weight and height")
    weight = st.number_input("Weight (kg)", min_value=1.0, step=0.1)
    height = st.number_input("Height (m)", min_value=0.1, step=0.01)
else:
    st.subheader("Enter your weight and height")
    weight_lbs = st.number_input("Weight (lbs)", min_value=1.0, step=0.1)  # Changed step to 0.1 for consistency
    feet = st.number_input("Height - feet", min_value=0, step=1)
    inches = st.number_input("Height - inches", min_value=0, max_value=11, step=1)
    height = ((feet * 12) + inches) * 0.0254
    weight = weight_lbs * 0.453592

if st.button("🧮 Calculate BMI"):
    if height > 0:
        bmi = weight / (height ** 2)
        bmi = round(bmi, 2)

        if bmi < 18.5:
            category = "Underweight 🦴"
            category_color = "blue"
        elif bmi < 25:
            category = "Normal ✅"
            category_color = "lightgreen"
        elif bmi < 30:
            category = "Overweight ⚠️"
            category_color = "yellow"
        else:
            category = "Obese 🔥"
            category_color = "orange"

        st.markdown(f"### 📊 Your BMI: **{bmi}**")
        st.markdown(f"<div style='background-color:{category_color}; padding: 10px; border-radius: 5px;'>**Category: {category}**</div>", unsafe_allow_html=True)

        st.session_state.history.append({
            "Weight (kg)": round(weight, 2),
            "Height (m)": round(height, 2),
            "BMI": bmi,
            "Category": category
        })

        category_clean = category.encode('ascii', 'ignore').decode()

        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=14)
        pdf.cell(200, 10, txt="BMI Report", ln=True, align="C")
        pdf.ln(10)
        pdf.cell(200, 10, txt=f"Weight: {weight:.2f} kg", ln=True)
        pdf.cell(200, 10, txt=f"Height: {height:.2f} m", ln=True)
        pdf.cell(200, 10, txt=f"BMI: {bmi}", ln=True)
        pdf.cell(200, 10, txt=f"Category: {category_clean}", ln=True)

        pdf_path = os.path.join(os.getcwd(), "bmi_report.pdf")
        pdf.output(pdf_path)

        with open(pdf_path, "rb") as f:
            st.download_button("⬇️ Download PDF Report", f, file_name="bmi_report.pdf", mime="application/pdf")

if st.session_state.history:
    with st.expander("📜 BMI History (this session)"):
        df = pd.DataFrame(st.session_state.history)
        st.dataframe(df, use_container_width=True)

with st.expander("📈 BMI Categories"):
    st.table(pd.DataFrame({
        "Category": ["Underweight", "Normal", "Overweight", "Obese"],
        "BMI Range": ["< 18.5", "18.5 – 24.9", "25 – 29.9", "30+"],
        "Emoji": ["🦴", "✅", "⚠️", "🔥"]
    }))
